# Mudcard
- **After going through the two lectures on feature importance, I'm curious about its application in large-scale datasets. Should we employ a global feature importance model to identify and select only the key features for use in the machine learning model, as a means to reduce computational demand? Or is it more appropriate to apply global feature importance analysis only after the machine learning algorithm has been executed? Especially for the large-scale dataset that has 100+ features.**
    - As long as you can work with your dataset on your laptop, it is not large scale, so a dataset with ~100 features is definitely not large by today's computational standards.
    - Global feature importances can only be calculate **after** you train a machine learning algorithm
    - Once you train an ML modl using all features and calculate the global importances, you could decide to drop certain features and re-train the model with a smaller number of features
- **In this lecture, we focused on using the SHAP method and visualizing the results of classification problems, does this method work for regression problems? And can the same sort of visualizations be used?**
    - Yep, it works similarly in regression models too
    - The only difference is that the SHAP array will only have two dimensions (n_points, n_ftrs)
    - We had a third dimension in classficiation, the number of classes but that's not necessary in regression because we predict one continuos target variable

# A note on imbalanced datasets
- we learnt that a classification problem is imbalanced if more than 90-95% of the points belong to one class (class 0) and only a small fraction of the points belong to the other class (class 1)
    - fraud detection
    - sick or not sick (usually by far most people are not sick)
- we learnt to not use a metric that relies on the True Negatives in the confusion matrix
    - no accuracy or ROC
    - use f_beta or the precision-recall curve instead

## What else can I do if I have an imbalanced dataset?
- most (but not all) classification algorithms we covered have a parameter called `class_weight` which allows you to assign more weight to the class 1 point
    - a misclassified class 1 point will contribute more to the cost function than a misclassified class 0 point
    - read the manual on `class_weight` because the different algorithms have slightly different definitions for this parameter
    - usually you can use `None`, `balanced`, or manually define what the class weight should be
    - it is worthwhile to tune this parameter if you have an imbalanced dataset
- resample/augment the dataset
    - SMOTE (Synthetic Minority Over-sampling Technique), see the [paper](https://arxiv.org/abs/1106.1813)
    - to improve the balance of the problem, new class 1 examples are synthesized from the existing examples
    - be careful though!
        - while resampling improves the balance of the dataset, the results of the model can be misleading
        - when you deploy the model, the incoming data will be as imbalanced as the original data

## Misleading results with resampling
- let's assume you have an imbalanced dataset with 99% of points in class 0 and 1% of points in class 1
- you resample it such that the improved class balance is 50-50
- here the confusion matrix of the trained model:

<table>
    <tr>
        <td colspan="2" rowspan="2"></td>
        <td colspan="2">Predicted class</td>			
    </tr>
    <tr>
        <td>Predicted Negative (0)</td>
        <td>Predicted Positive (1)</td>
    </tr>
    <tr>
        <td rowspan="2">Actual class</td>
        <td>Condition Negative (0)</td>
        <td><b>True Negative (TN): 45%</b></td>
        <td><b>False Positive (FP): 5%</b></td>
    </tr>
    <tr>
        <td>Condition Positive (1)</td>
        <td><b>False Negative (FN): 5%</b></td>
        <td><b>True Positive (TP): 45%</b></td>
    </tr>
</table>

- 90% accuracy which is well above the 50% baseline accuracy!
- the precision, recall, and f1 scores are all 0.9.
- it looks great, doesn't it?
- let's rewrite the confusion matrix to reflect rates with respect to the Condition Negative and Condition Positive points!

<table>
    <tr>
        <td colspan="2" rowspan="2"></td>
        <td colspan="2">Predicted class</td>			
    </tr>
    <tr>
        <td>Predicted Negative (0)</td>
        <td>Predicted Positive (1)</td>
    </tr>
    <tr>
        <td rowspan="2">Actual class</td>
        <td>Condition Negative (0) 50% of the points</td>
        <td><b>90% of CNs are correctly classified</b></td>
        <td><b>10% of CNs are incorrectly classified</b></td>
    </tr>
    <tr>
        <td>Condition Positive (1) 50% of the points</td>
        <td><b>10% of CPs are incorrectly classified</b></td>
        <td><b>90% of CPs are correctly classified</b></td>
    </tr>
</table>

## Let's deploy this model

- the incoming data has the same balance as the original dataset (99% to 1%)
- let's assume we have 1e5 new points, 9.9e4 belongs to class 0, 1000 belongs to class 1
- what will be the numbers in the confusion matrix?


<table>
    <tr>
        <td colspan="2" rowspan="2"></td>
        <td colspan="2">Predicted class</td>			
    </tr>
    <tr>
        <td>Predicted Negative (0)</td>
        <td>Predicted Positive (1)</td>
    </tr>
    <tr>
        <td rowspan="2">Actual class</td>
        <td>Condition Negative (0) 99000 points</td>
        <td><b>True Negative (TN): 99000 * 0.9 = 89100 </b></td>
        <td><b>False Positive (FP): 99000 * 0.1 = 9900 </b></td>
    </tr>
    <tr>
        <td>Condition Positive (1) 1000 points</td>
        <td><b>False Negative (FN): 1000 * 0.1 = 100</b></td>
        <td><b>True Positive (TP): 1000 * 0.9 = 900</b></td>
    </tr>
</table>

- the accuracy of this model is still 0.90 but now it is well below the baseline of 0.99!
- recall is good (0.90) but the precision is not great (~0.083)
- the f1 score is ~0.15
- the false positives are overwhelming
- this is why you need to be careful with resampling

## Quiz

## Mudcard